In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
for filename in filenames:
print(os.path.join(dirname, filename))
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import zipfile
def unzip_files(directory_path, extract_to_path):
for filename in os.listdir(directory_path):
if filename.endswith('.zip'):
zip_file_path = os.path.join(directory_path, filename)
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
zip_ref.extractall(extract_to_path)
main_directory = '/kaggle/input/dogs-vs-cats-redux-kernels-edition'
extract_to_path = '/kaggle/working/'
unzip_files(main_directory, extract_to_path)

In [ ]:
import glob
import matplotlib.pyplot as plt
import cv2
import keras
from tensorflow.keras.callbacks import EarlyStopping
from keras import optimizers
import tensorflow as tf
from keras.layers import Dense,Flatten,Dropout,BatchNormalization
from keras import optimizers
from keras.models import Sequential
from keras.losses import BinaryCrossentropy
from keras.activations import sigmoid
from keras.applications import ResNetRS420
import tensorflow as tf
from keras.layers import Dense,Flatten,Dropout,BatchNormalization
from keras.applications import DenseNet121,DenseNet169,DenseNet201,RegNetY320,RegNetX320

In [ ]:
train_dir= '/kaggle/working/train'
train_data=glob.glob(train_dir+'**/*.jpg',root_dir=train_dir)
train_data=train_data[:5000]

In [ ]:
X_train=[]
for img_path in train_data:
img= cv2.imread(img_path)
img= cv2.resize(img,(256,256))
img= img/255
X_train.append(img)
X_train= np.array(X_train)

In [ ]:
X_train.shape

In [ ]:
files_name= glob.glob('*.jpg',root_dir= train_dir)
files_name[:5]

In [ ]:
y_train = [file_name.split('.')[0] for file_name in files_name]

In [ ]:
y_train = np.array(y_train)
y_train[:5]

In [ ]:
y_train= y_train[:5000]

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder= LabelEncoder()
y_train= encoder.fit_transform(y_train)

In [ ]:
plt.figure(figsize=(10,10))
for i in range(min(16, len(X_train))):
plt.subplot(4, 4, i+1)
plt.imshow(X_train[i])
plt.title(y_train[i])
plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
# Freeze the layers of the base model
for layer in base_model.layers:
layer.trainable = False
model = tf.keras.models.Sequential([
base_model,
BatchNormalization(),
Flatten(),
Dense(256,activation='relu'),
BatchNormalization(),
Dense(128,activation='relu'),
Dense(64,activation='relu'),
Dropout(0.5),
Dense(1,activation='sigmoid'),
])

In [ ]:
model.summary()

In [ ]:
# Learning rate schedule
lr_schedule = optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.00001,
    decay_steps=2000,
    decay_rate=0.7
)
optimizer = optimizers.Adam(learning_rate=lr_schedule)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=30, verbose=1, validation_split=0.2, callbacks=[early_stopping])

In [ ]:
# Plot training and validation accuracy
plt.figure(figsize=(12, 4))
# Plot training accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
# Plot training and validation loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
model.save('catvsdog.keras')

In [ ]:
from keras.models import load_model
model= load_model('/kaggle/working/catvsdog.keras')

In [ ]:
test_dir= '/kaggle/working/test'

In [ ]:
test_data=glob.glob(test_dir+'/*.jpg')

In [ ]:
# X_test=[]
# for img_path in test_data:
# img= cv2.imread(img_path)
# img= cv2.resize(img,(256,256))
# img= img/255
# X_test.append(img)
# X_test= np.array(X_test)

In [ ]:
import numpy as np
from keras.models import load_model
from keras.preprocessing import image
import cv2
# Define a function to preprocess images
def preprocess_image(img):
img = cv2.resize(img, (256, 256))
img = img / 255.0 # Normalize pixel values
img = np.expand_dims(img, axis=0)
return img
# Example of batch processing
batch_size = 32
# Initialize an empty list to store predictions
all_predictions = []
for i in range(0, len(test_data), batch_size):
batch_paths = test_data[i:i + batch_size]
batch_images = [cv2.imread(img_path) for img_path in batch_paths]
batch_images = [preprocess_image(img) for img in batch_images]
# Convert the batch to a numpy array
batch_data = np.vstack(batch_images)
# Predictions for the current batch
predictions = model.predict(batch_data)
# Extend the predictions list
all_predictions.extend(predictions.flatten())
# Now, 'all_predictions' contains predictions for all test images
all_predictions = np.array(all_predictions)